Feature selection reduces the dimensionalit of data for the following reasons:
- Reduces overfitting by removing noise introduced by some of the features
- Reduces training time, which allows you to experiment more with different models and hyperparameters
- Reduces data acquisition requirements
- Improves comprehensibility of the model because a smaller set of features is more comprehendible to humans. This enables us to focus on the main sources of predictability

Feature selection methods generally fall into 2 categories. Filter Methods and Wrapper Methods. 

- Filter Methods: Apply a statistical measure and assign a score to each feature one at a time. Pearson's X2 and ANOVA F-Value based feature selection. 

- Wrapper Methods: Use a subset of features. Based on the results drawn from the previous model trained on that subset of features, they are either added or removed from the subset. The problem is essentially reduced to a search problem. Greedy algos (https://en.wikipedia.org/wiki/Greedy_algorithm) are the most desirable in multivariate feature selection scenarios because the wrapper methods are usually computationally very expensive and greedy algos don't necessarily provide the optimal solution, which is a good thing because it makes them less prone to overfitting. Forward Selection, Backward Elimination, Recursive Feature Elimination. 

In [97]:
import os, sys
import numpy as np
import pandas as pd
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import mutual_info_classif
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_selection import RFE
from sklearn.base import clone
import itertools
from sklearn.preprocessing import LabelBinarizer
from scipy.stats import f as f_distribution

In [98]:
datasource = "datasets/winequality-red.csv"
print(os.path.exists(datasource))

True


In [99]:
df = pd.read_csv(datasource).sample(frac = 1).reset_index(drop = True)
del df["Unnamed: 0"]
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,6.2,0.56,0.09,1.7,0.053,24.0,32.0,0.99402,3.54,0.60,11.3,5
1,7.0,0.36,0.21,2.3,0.086,20.0,65.0,0.99558,3.40,0.54,10.1,6
2,7.3,0.48,0.32,2.1,0.062,31.0,54.0,0.99728,3.30,0.65,10.0,7
3,8.4,0.36,0.32,2.2,0.081,32.0,79.0,0.99640,3.30,0.72,11.0,6
4,7.1,0.53,0.07,1.7,0.071,15.0,24.0,0.99510,3.29,0.66,10.8,6


In [100]:
X = np.array(df.iloc[:, :-1])
y = np.array(df["quality"])

## Feature selection solution space

From algorithm analysus' point of view, a solution for feature selection problems can be represented as a boolean vector, each component indicating whether the corresponding feature has been selected.

In [101]:
selected = np.array([False, True, True, False, False, True, True, False, False, False, True])
print(selected)

[False  True  True False False  True  True False False False  True]


scikit-learn calls the corresponding indices to feature columns selected "support", which can be obtained using np.flatnonzero(). 

https://docs.scipy.org/doc/numpy/reference/generated/numpy.flatnonzero.html

Return indices that are non-zero in the flattened version of a.


In [102]:
support = np.flatnonzero(selected)
print(support)

[ 1  2  5  6 10]


Thus, a naive approach that exhaustively search all subsets of features would have to verify 2^p solutions. This is very inefficient. However, we will run an exhaustive search for all solutions that provide 5 features to establish a baseline. This limits our time complexity. 

In [103]:
def search_combinations(estimator, X, y, k = 5):
    # fit and score model based on some subset of features
    score = lambda X_features: clone(estimator).fit(X_features, y).score(X_features, y)
    
    # enumerate all combinations of 5 features
    for subset in itertools.combinations(range(X.shape[1]), 5):
        yield score(X[:, subset]), subset

In [104]:
scores = search_combinations(LinearRegression(), X, y)

# feed it a model, X, and y. 
# it'll iterate through all possible variations (up to 5)
# and fit/score on those variations

In [105]:
sorted(scores, reverse = True)[:5]

[(0.35149423850184047, (1, 4, 6, 9, 10)),
 (0.34831403567393349, (1, 4, 8, 9, 10)),
 (0.34695788821029638, (1, 6, 8, 9, 10)),
 (0.34667490118495603, (0, 1, 4, 9, 10)),
 (0.34580097696162626, (0, 1, 6, 9, 10))]

## Wrapper Methods

### Forward Selection

Forward selection is an iterative method in which we start with having no feature in the model. In each iteration, we keep adding the feature which best improves our model

In [106]:
def forward_select(estimator, X, y, k = 2):
    # this array holds indicators of whether each feature is currently selected
    selected = np.zeros(X.shape[1]).astype(bool)
    
    # fit and score the model based on some subset of features
    score = lambda X_features: clone(estimator).fit(X_features, y).score(X_features, y)
    
    # find indices to selected columns
    selected_indices = lambda: list(np.flatnonzero(selected))
    
    # repeated til k features are selected
    while np.sum(selected) < k:
        # indices to unselected column
        rest_indices = list(np.flatnonzero(~selected))
        
        # compute model scores with an additional feature
        scores = [score(X[:, selected_indices() + [i]]) for i in rest_indices]
        
        print("\n * accuracy if adding column: \n    ", {i: int(s * 100) for i, s in zip(rest_indices, scores)})
        
        # find index within "rest_indices" that points to the most predictive feature not yet selected
        idx_to_add = rest_indices[np.argmax(scores)]
        print("add column", idx_to_add)
        
        # select this new feature
        selected[idx_to_add] = True
        print("================================")
        
    return selected_indices()

In [107]:
support = sorted(forward_select(LinearRegression(), X, y))


 * accuracy if adding column: 
     {0: 1, 1: 15, 2: 5, 3: 0, 4: 1, 5: 0, 6: 3, 7: 3, 8: 0, 9: 6, 10: 22}
add column 10

 * accuracy if adding column: 
     {0: 25, 1: 31, 2: 25, 3: 22, 4: 22, 5: 22, 6: 23, 7: 23, 8: 25, 9: 26}
add column 1


## Backwards Elimination

In backwards elimination, we basically just do the opposite. We start with ALL the features and remove the least significant feature at each iteration

In [108]:
def backwards_eliminate(estimator, X, y, k = 5):
    # this array holds indicators of whether each feature is currently selected
    selected = np.ones(X.shape[1]).astype(bool)
    
    # fit and score model based on some subset of features
    score = lambda X_features: clone(estimator).fit(X_features, y).score(X_features, y)
    
    # find indices to selected columns
    selected_indices = lambda: list(np.flatnonzero(selected))
    
    # repeat til k features are selected
    while np.sum(selected) > k:
        # compute model scores with one of the features removed
        scores = [score(X[:, list(set(selected_indices()) - {i})]) for i in selected_indices()]
        print("\n accuracy if removing column: \n", {i: int(s*100) for i, s in zip(selected_indices(), scores)})
        
        # find index that points to the least predictive feature
        idx_to_remove = selected_indices()[np.argmax(scores)]
        print("remove column", idx_to_remove)
        
        # remove this feature
        selected[idx_to_remove] = False
        print("================================")
        
    return selected_indices()

In [109]:
support = sorted(backwards_eliminate(LinearRegression(), X, y))


 accuracy if removing column: 
 {0: 36, 1: 32, 2: 35, 3: 36, 4: 35, 5: 35, 6: 35, 7: 36, 8: 35, 9: 33, 10: 31}
remove column 7

 accuracy if removing column: 
 {0: 36, 1: 32, 2: 35, 3: 36, 4: 35, 5: 35, 6: 35, 8: 35, 9: 33, 10: 24}
remove column 0

 accuracy if removing column: 
 {1: 32, 2: 35, 3: 35, 4: 35, 5: 35, 6: 35, 8: 35, 9: 33, 10: 24}
remove column 3

 accuracy if removing column: 
 {1: 32, 2: 35, 4: 35, 5: 35, 6: 35, 8: 35, 9: 33, 10: 24}
remove column 2

 accuracy if removing column: 
 {1: 31, 4: 34, 5: 35, 6: 34, 8: 35, 9: 33, 10: 24}
remove column 5

 accuracy if removing column: 
 {1: 31, 4: 34, 6: 34, 8: 35, 9: 33, 10: 23}
remove column 8


## Recursive Feature Elimination

Recursive feature elimination is an even more greedy algo. It finds good performing feature subset with high efficiency. The importance of each feature is obtained either through a coef attribute or through a feature_importances attribute. So in order for recursive feature elimination to work, the model is required to provide either of these attributes. 

We typically start off using a low complexity model and use it as a benchmark for feature selection

In [110]:
model = LinearRegression()
selector = RFE(model, 5)
selector.fit(X, y)

RFE(estimator=LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False),
  n_features_to_select=5, step=1, verbose=0)

In [111]:
print("Number of Features:", selector.n_features_)

Number of Features: 5


In [112]:
print("Selected Features:", np.flatnonzero(selector.support_))

Selected Features: [1 4 7 8 9]


Then we transform the dataset to include only these features

In [113]:
X_new = selector.transform(X)
print(X_new.shape)

(1599, 5)


## Filter Methods

These methods rank feature predictiveness one by one, as opposed to considering a subset. They incorporate statistical methods to rank each feature instead of measuring accuracy of a model trained on selected features.

### Peason's X^2 test based feature selection
The following constructs the approximate X^2 distribution and scores each feature vs the label in order to determine which feature is more relevant, one at a time...and then selects features according to the score. 

In [114]:
selector = SelectKBest(chi2, k = 5)

In [115]:
selector.fit(X, y)

SelectKBest(k=5, score_func=<function chi2 at 0x00000181D77B2158>)

In [116]:
print("X^2 statistic: \n", selector.scores_)

X^2 statistic: 
 [  1.12606524e+01   1.55802891e+01   1.30256651e+01   4.12329474e+00
   7.52425579e-01   1.61936036e+02   2.75555798e+03   2.30432045e-04
   1.54654736e-01   4.55848775e+00   4.64298922e+01]


In [117]:
print("Selected indices: \n", selector.get_support(True))

Selected indices: 
 [ 1  2  5  6 10]


We'll then call transform() to select those feature columns from the dataset and store them into a new variable called X_selected

In [118]:
X_selected = selector.transform(X)
X_selected.shape

(1599, 5)

Or we can slice and dice. This does the same thing as transform

In [119]:
np.allclose(X_selected, X[:, [1, 2, 5, 6, 10]])

True

In [120]:
X[:, [1, 2, 5, 6, 10]]

array([[  0.56,   0.09,  24.  ,  32.  ,  11.3 ],
       [  0.36,   0.21,  20.  ,  65.  ,  10.1 ],
       [  0.48,   0.32,  31.  ,  54.  ,  10.  ],
       ..., 
       [  0.29,   0.36,  35.  ,  53.  ,  12.4 ],
       [  0.31,   0.51,  14.  ,  28.  ,   9.8 ],
       [  0.44,   0.2 ,  24.  ,  64.  ,  11.7 ]])

Let's take a closer look at this procedure so we can better distinguish different feature selection methods. X^2 test has many applications. For feature selection, we utilize X^2 statistic to test for dependence of each feature towards determining label. 

##### Step 1: Encode labels into orthogonal vector space
This is also known as one-hot encoding. It's applicable to classification problems. Consider an example where you have defined 3 categories for possible outcomes: A/B/C. In order for machine learning algos to be able to handle this type of data, we have to convert them to numbers. One-hot encoding uses a vector (y1, y2, y3) where yi = [result falls into the ith category]. 

Therefore, one-hot encoding for A, B, and C categories becomes (1, 0, 0), (0, 1, 0), and (0, 0, 1) respectively. This has an advantage over plainly translating A, B, C into 1, 2, 3 (aka sparse encoding) in a way that orthogonal vectors do not impose assumptions of their order or magnitudes between categories like numbers would. 

For example, 3 > 1 is true. However, it doesn't mean to imply that C > A or C is superior to A in any way. However, this would affect the model's numerical stability. 

Therefore, one-hot encoding is a widely adopted technique for processing categories. Sparse encoding could be used when persisting a dataset in order to save storage space. One-hot encoding is how the categorical/nominal variables are encoded as independent predictors in the regression formula. 

In [121]:
Y = np.array(LabelBinarizer().fit_transform(y))

In [122]:
print(Y.shape)

(1599, 6)


In [123]:
print(Y[:5])

[[0 0 1 0 0 0]
 [0 0 0 1 0 0]
 [0 0 0 0 1 0]
 [0 0 0 1 0 0]
 [0 0 0 1 0 0]]


#### Step 2: Compute the contingency table of observed frequencies
"Observed" is a (# classes) - by - (# features) matrix that contains the "number of occurrences" for each combination of feature and classes. 

In [124]:
observed = np.dot(Y.T, X)

In [125]:
observed

array([[  8.36000000e+01,   8.84500000e+00,   1.71000000e+00,
          2.63500000e+01,   1.22500000e+00,   1.10000000e+02,
          2.49000000e+02,   9.97464000e+00,   3.39800000e+01,
          5.70000000e+00,   9.95500000e+01],
       [  4.12300000e+02,   3.67800000e+01,   9.23000000e+00,
          1.42800000e+02,   4.80600000e+00,   6.50000000e+02,
          1.92100000e+03,   5.28167500e+01,   1.79220000e+02,
          3.16100000e+01,   5.44050000e+02],
       [  5.56190000e+03,   3.92965000e+02,   1.65950000e+02,
          1.72215000e+03,   6.31530000e+01,   1.15660000e+04,
          3.84860000e+04,   6.79027570e+02,   2.25067000e+03,
          4.22880000e+02,   6.74170000e+03],
       [  5.32550000e+03,   3.17395000e+02,   1.74700000e+02,
          1.58045000e+03,   5.42020000e+01,   1.00240000e+04,
          2.60750000e+04,   6.35840410e+02,   2.11693000e+03,
          4.30860000e+02,   6.78163333e+03],
       [  1.76560000e+03,   8.03800000e+01,   7.46600000e+01,
          5.41

#### Step 3: Compute the expected frequencies using marginal frequencies
"Expected" will have the same shape as "Observed" matrix but represents the expected frequencies in theory

In [126]:
expected = np.dot(Y.mean(axis = 0).reshape(1, -1).T, # mean value for all classes (transposed) 
                 X.sum(axis = 0).reshape(1, -1)) # marginal frequencies for all features

#### Step 4: Compute the X^2 statistic between observed and expected

In [127]:
chi_squared = np.sum((observed - expected) ** 2 / expected, axis= 0)
print(chi_squared)

[  1.12606524e+01   1.55802891e+01   1.30256651e+01   4.12329474e+00
   7.52425579e-01   1.61936036e+02   2.75555798e+03   2.30432045e-04
   1.54654736e-01   4.55848775e+00   4.64298922e+01]


Compare against sklearn's chi2(), which returns 2 arrays. X^2 statistic and p-value

In [128]:
chi2_sklearn, pvalue_sklearn = chi2(X, y)

In [129]:
print("Chi Squared Statistic: \n", chi2_sklearn)

Chi Squared Statistic: 
 [  1.12606524e+01   1.55802891e+01   1.30256651e+01   4.12329474e+00
   7.52425579e-01   1.61936036e+02   2.75555798e+03   2.30432045e-04
   1.54654736e-01   4.55848775e+00   4.64298922e+01]


In [130]:
print("P values: \n", pvalue_sklearn)

P values: 
 [  4.64500416e-02   8.15035154e-03   2.31394417e-02   5.31804675e-01
   9.79968040e-01   3.82728810e-33   0.00000000e+00   1.00000000e+00
   9.99526491e-01   4.72096321e-01   7.42403757e-09]


#### Step 5: Rank features descendingly by X^2 statistic
Optionally, then sort these indices so they remain in order

In [131]:
support = sorted(np.argsort(-chi_squared)[0:5])
print(support)

[1, 2, 5, 6, 10]


#### Step 6: Select these features and transform the dataset

In [132]:
X_new = X[:, support]
np.allclose(X_new, X_selected)

True

### About the p value
The p value is very useful because it could tell you quantitavely how probable each feature is relevant, which in turns helps you decide how many and which features are worthwhile to retain. Our null hypothesis is that a class label is independent of a feature. Since we used X^2 statistic for testing what the null hypotheis has claimed (independence between features and labels). We can choose a p value > 2.5% as the critical level at which the null hypothesis can not be rejected and reject the hypothesis otherwise. In other words, if the feature has more than 10% chance to be independent of the class, then it is not a good predictor. The alternative hypothesis is that the feature is a probable predictor of the class (aka the class is dependent on the feature).

<table>
<tr><td><strong>feature idx</strong></td><td>0</td><td>1</td><td>2</td><td>3</td><td>4</td><td>5</td><td>6</td><td>7</td><td>8</td><td>9</td><td>10</td></tr>
<tr><td><strong>χ²</strong></td><td>11.3</td><td>15.6</td><td>13.0</td><td>4.12</td><td>0.752</td><td>162</td><td>2.76e+03</td><td>2.30e-04</td><td>0.155</td><td>4.56</td><td>46.4</td></tr>
<tr><td><strong>p-value</strong></td><td>~5%</td><td>~0.5%</td><td>~1%</td><td>90%~95%</td><td>100%</td><td>0%</td><td>0%</td><td>100%</td><td>100%</td><td>90%~95%</td><td>0%</td></tr>
<tr><td><strong>interpretation</strong></td><td>independent</td><td><strong>dependent</strong></td><td><strong>dependent</strong></td><td>independent</td><td>independent</td><td><strong>dependent</strong></td><td><strong>dependent</strong></td><td>independent</td><td>independent</td><td>independent</td><td><strong>dependent</strong></td></tr>
</table>

So we should select feature 1, 2, 5, 6 and 10.

In [133]:
help(chi2)

Help on function chi2 in module sklearn.feature_selection.univariate_selection:

chi2(X, y)
    Compute chi-squared stats between each non-negative feature and class.
    
    This score can be used to select the n_features features with the
    highest values for the test chi-squared statistic from X, which must
    contain only non-negative features such as booleans or frequencies
    (e.g., term counts in document classification), relative to the classes.
    
    Recall that the chi-square test measures dependence between stochastic
    variables, so using this function "weeds out" the features that are the
    most likely to be independent of class and therefore irrelevant for
    classification.
    
    Read more in the :ref:`User Guide <univariate_feature_selection>`.
    
    Parameters
    ----------
    X : {array-like, sparse matrix}, shape = (n_samples, n_features_in)
        Sample vectors.
    
    y : array-like, shape = (n_samples,)
        Target vector (class labels)

Let's generate a X^2 table

In [134]:
def chi2_table(degree_of_freedoms):
    from scipy.stats import chi2 as chi2_distribution
    
    pvalue = np.array([0.995, 0.99, 0.975, 0.95, 0.90, 0.10, 0.05, 0.025, 0.01, 0.005])
    return pd.DataFrame(chi2_distribution.isf(pvalue, 
                                              np.expand_dims(degree_of_freedoms, 1)), 
                                              columns = pvalue, index = degree_of_freedoms)

    # isf(p) = inverse(1 - cdf)(p) which takes p value returns chi square value
    # where cdf is short for cumulative distribution function

In [135]:
chi2_table(range(5, 12))

,0.995,0.99,0.975,0.95,0.9,0.1,0.05,0.025,0.01,0.005
5,0.411742,0.554298,0.831212,1.145476,1.610308,9.236357,11.070498,12.832502,15.086272,16.749602
6,0.675727,0.872090,1.237344,1.635383,2.204131,10.644641,12.591587,14.449375,16.811894,18.547584
7,0.989256,1.239042,1.689869,2.167350,2.833107,12.017037,14.067140,16.012764,18.475307,20.277740
8,1.344413,1.646497,2.179731,2.732637,3.489539,13.361566,15.507313,17.534546,20.090235,21.954955
9,1.734933,2.087901,2.700389,3.325113,4.168159,14.683657,16.918978,19.022768,21.665994,23.589351
10,2.155856,2.558212,3.246973,3.940299,4.865182,15.987179,18.307038,20.483177,23.209251,25.188180
11,2.603222,3.053484,3.815748,4.574813,5.577785,17.275009,19.675138,21.920049,24.724970,26.756849


### ANOVA F-value based features selection

The following calculates Pearson's correlation of each feature vs the label in order to determine which feature is more relevant, one at a time, then selects features according to the ANOVA F-Value derived from Pearson's correlation. 

https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.pearsonr.html



In [136]:
selector = SelectKBest(f_regression, k = 5)
selector.fit(X, y)

SelectKBest(k=5, score_func=<function f_regression at 0x00000181D77B21E0>)

In [137]:
print("Score: \n", selector.scores_)

Score: 
 [  2.49600375e+01   2.87444450e+02   8.62577262e+01   3.01183699e-01
   2.69856084e+01   4.10850227e+00   5.66578176e+01   5.04052231e+01
   5.34046221e+00   1.07740433e+02   4.68267011e+02]


In [138]:
print("Selected Indices: \n", selector.get_support(True))

Selected Indices: 
 [ 1  2  6  9 10]


In [139]:
X_new = selector.transform(X)
print(X_new)

[[  0.56   0.09  32.     0.6   11.3 ]
 [  0.36   0.21  65.     0.54  10.1 ]
 [  0.48   0.32  54.     0.65  10.  ]
 ..., 
 [  0.29   0.36  53.     1.01  12.4 ]
 [  0.31   0.51  28.     0.93   9.8 ]
 [  0.44   0.2   64.     0.57  11.7 ]]


F test can be used as hypothesis testing for a ratio of two X^2 distributions. The F-test for correlation coefficients is derived from testing a ratio between regression sum square (SSR) and squared sum error (SSE). 

$$ SSR = \sum _{i=1}^n {\left(\hat {y_i} - \bar y \right)^2}$$

$$ SSE = \sum _{i=1}^n {\left(\hat {y_i} - y_i \right)^2}$$

$$ F  = \frac{SSR/(v-1) }{SSE/(n-2)} $$

With correlation coefficients, this is 

$$ F = \frac{r^2/1 }{(1-r^2)/(n-2)} $$

with degree of freedom 1 and (n-2) respectively.

Its null hypothesis claims r = 0, i.e. the independence of each feature and label.
We could theoretically make hypothesis testing using an F-distribution table.

Here's how to generate F table.

In [140]:
def f_table(alpha, v1, v2):
    from scipy.stats import f as f_distribution
    v1 = np.array(list(v1))
    v2 = np.array(list(v2))
    return pd.DataFrame(f_distribution.isf(alpha, v1[np.newaxis, ...], v2[..., np.newaxis]),
                           columns = v1, index = v2)

In [141]:
f_table(0.1, range(1, 9), itertools.chain(range(1,10), range(10, 260, 20)))

,1,2,3,4,5,6,7,8
1,39.863458,49.500000,53.593245,55.832961,57.240077,58.204416,58.905953,59.438981
2,8.526316,9.000000,9.161790,9.243416,9.292626,9.325530,9.349081,9.366770
3,5.538319,5.462383,5.390773,5.342644,5.309157,5.284732,5.266195,5.251671
4,4.544771,4.324555,4.190860,4.107250,4.050579,4.009749,3.978966,3.954940
5,4.060420,3.779716,3.619477,3.520196,3.452982,3.404507,3.367899,3.339276
6,3.775950,3.463304,3.288762,3.180763,3.107512,3.054551,3.014457,2.983036
7,3.589428,3.257442,3.074072,2.960534,2.883344,2.827392,2.784930,2.751580
8,3.457919,3.113118,2.923796,2.806426,2.726447,2.668335,2.624135,2.589349
9,3.360303,3.006452,2.812863,2.692680,2.610613,2.550855,2.505313,2.469406
10,3.285015,2.924466,2.727673,2.605336,2.521641,2.460582,2.413965,2.377150


X^2 and F-value based feature selection work better on classification and regression respectively. The general steps for feature selection with sklearn was:
1) Choose a feature scoring method
2) Initialize a feature selector
3) Fit feature selector on the data

Other feature selection methods provided by sklearn include:
* Classification: chi2, f_classif, mutual_info_classif
* Regression: f_regression, mutual_info_regression

Wrapper methods are computationally expensive. Greedy algos don't necessary provide the optimal solution (which may be good because it makes them less prone to overfitting)